# note: This program needs openAI API

# Prep dataset

In [ ]:
import pandas as pd
import numpy as np
import os
import scipy
import re

os.getcwd()

In [ ]:
data_foloder = 'your folder'

baseline = pd.read_csv(data_foloder + 'test.csv')

baseline.head() 

# Install lib and dependencies

In [ ]:
!pip install transformers # no need to install every time you open it

In [3]:
import transformers
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

In [4]:
from openai.embeddings_utils import get_embeddings, cosine_similarity # important
import openai
openai.api_key = 'your key'

# Get sent embeddings from GPT3 response embeddings 3df

In [ ]:
buglist = {}
def calc_gpt3(r):  # get embeddings
  if r[0] % 2 != 0: # progress: print even grid
    print('current grid: ', r[0])
  if type(r['content']) == str and r['n_words'] < 2048: # max for GPT3
    temp = r['content'].split(' ') 
    input = ' '.join([i for i in temp if (i != '') & ('{' not in i) & ('}' not in i) & ('#' not in i)]) # triple check: exclude NSV and the restart symbol
    try:
      # split by . or ? or !
      input.replace('?', '.').replace('!', '.')
      vec = get_embeddings(input.split(' . '), engine = 'your engine') # take in the whose response as a list of sentence, sep by . ! ?
    except:
      buglist[r[0]] = input.split(' ') # store grid
    return vec
  else:
    return np.nan

# double check your input df
# make sure there is no '  ' or '   '

dfs = [baseline]
temp = -1
for df in dfs:
  temp += 1
  df = df[df['grid'] == 'test000'] # only select 1 individual for demo purpose
  df["gpt3_embed"] = '' # create a new empty column
  df["gpt3_embed"] = df.apply(lambda r: calc_gpt3(r), axis = 1) # apply the embedding function to the dataframe
  df.to_csv(str(temp) + '_sent.csv') 
df.head()

In [36]:
buglist

{}

In [37]:
len(df['gpt3_embed'][1]) # the number of sentences in that response

13

In [38]:
len(df['gpt3_embed'][1][3]) # every sentence get a len (i.e. dimension) 2048 embedding vector

2048

# FOC and SOC

In [39]:
# stats ignoring nan
from numpy import nanmedian

import scipy
def iqr(x):
  return scipy.stats.iqr(np.array(x), nan_policy='omit')

from numpy import quantile
def q5(x):
    return np.quantile(np.array(x), 0.05)

def q95(x):
    return np.quantile(np.array(x), 0.95)

In [ ]:
temp_file = 0 # give df a temp name; the intermediate df will be discarded bcs otherwise it occupies too much space
cs=['foc', 'soc']
stats = ['_median', '_iqr', '_q5', '_q95']
dfs = [df]
for df in dfs:
    temp += 1
    # create new empty columns
    for c in cs:
        for stat in stats:
            cur = 'gpt3_' + c + stat
            df[cur] = ''
df.head()

In [ ]:
temp_file = -1 # keep track of file names
for df in dfs:
    temp_file += 1
    df['gpt3_foc_similarity'] = ''
    df['gpt3_soc_similarity'] = ''
    # loop over each response
    for i in df.index:
        temp_foc = [] # for each individual, store the list of cos similarity
        temp_soc = []
        # calculate average similarity for sentence pairs, either adjacent or with one intervening
        for idx, sent in enumerate(df['gpt3_embed'][i]):
            try:
                temp_foc.append(cosine_similarity(sent, df['gpt3_embed'][i][idx+1])) # get a list of similarities for that response
                
                df['gpt3_foc_similarity'][i] = temp_foc # record intermediate similarities 
                
                df['gpt3_foc_median'][i] = np.nanmedian(temp_foc) # add more stats here
                df['gpt3_foc_iqr'][i] = iqr(temp_foc)
                df['gpt3_foc_q5'][i] = q5(temp_foc)
                df['gpt3_foc_q95'][i] = q95(temp_foc)

                temp_soc.append(cosine_similarity(sent, df['gpt3_embed'][i][idx+2]))

                df['gpt3_soc_similarity'][i] = temp_soc # record intermediate similarities 

                df['gpt3_soc_median'][i] = np.nanmedian(temp_soc)
                df['gpt3_soc_iqr'][i] = iqr(temp_soc)
                df['gpt3_soc_q5'][i] = q5(temp_soc)
                df['gpt3_soc_q95'][i] = q95(temp_soc)

            except IndexError:
                continue
    df.to_csv(str(temp_file) + '_sent.csv')
df.head()